# Elements of Programming Interviews
## Binary Trees
### Track 6: 10.1, 10.2, 10.4, 10.8, 10.10, 10.11, 10.14

### 10.1 - Test If a Binary Tree Is Balanced
>A binary tree is said to be balanced if for each node in the tree, the difference in the height of it's left and right subtrees is at most one. A perfect binary tree is balanced, as is a complete binary tree.
<img src='Images/balanced_tree.png'>

In [1]:
class BT(object):
    def __init__(self, data, left=None, right=None):
        self.data = data
        self.left = left
        self.right = right
balanced_tree = BT(314, BT(6, BT(271, BT(28), BT(0)), BT(561, right=BT(3, BT(17)))), BT(6, BT(2, right=BT(1, BT(401,right=BT(641)), BT(257))), BT(271, right=BT(28))))
unbalanced_tree = BT(1, BT(2, BT(3, BT(4))), BT(5))
#Unbalanced Tree
#        1
#       2 5
#      3
#     4

In [10]:
def post_order(node):
    if node is not None:
        post_order(node.left)
        post_order(node.right)
        print node.data,
    return
post_order(unbalanced_tree)

4 3 2 5 1


In [11]:
def height(node):
    if node is None:
        return -1
    return max(height(node.left), height(node.right)) + 1

In [13]:
def is_balanced(root):
    height_diff = abs(height(root.left) - height(root.right))
    return height_diff <= 1

In [14]:
is_balanced(unbalanced_tree), is_balanced(balanced_tree)

(False, True)